# Salient Oddities

### Wrangling for percent change

When you think of this R code in the context of table transformations, it does some pretty interesting things to calculate percent change between years. The `lag` function in R kind of "shifts" a vector in order to compute what is an inter-column calculation.

```{r}
totals.pch <- totals %>% mutate(other_perc = grn_perc + lib_perc + oth_perc, 
                                other = grn + lib + oth) %>% 
                         select(-grn_perc, -lib_perc, -oth_perc) %>%
                         group_by(county) %>% 
                         filter(date == '2014-09-30' | 
                                date == '2016-09-30' | 
                                date == '2018-09-30') %>% 
                         arrange(county, date) %>% 
                         mutate(pch_16_dem = (dem/lag(dem) - 1) * 100, 
                                pch_16_rep = (rep/lag(rep) - 1) * 100, 
                                pch_16_unaf = (unaf/lag(unaf) - 1) * 100,
                                pch_16_other = (other/lag(other) - 1) * 100,
                                pch_14_dem = (dem/lag(dem, 2) - 1) * 100, 
                                pch_14_rep = (rep/lag(rep, 2) - 1) * 100, 
                                pch_14_unaf = (unaf/lag(unaf, 2) - 1) * 100,
                                pch_14_other = (other/lag(other, 2) - 1) * 100) %>% 
  filter(date == '2018-09-30') %>%
  ungroup() %>%
  select(-conf.mailing, -inactive)
```

# Reshaping via self-joins

```
candidate_pairs = (
    contributor_totals
    .rename(columns = {
        "Candidate Name": "candidate"
    })
    [[
        "donor_id",
        "candidate"
    ]]
    .pipe(lambda df: (
        df
        .merge(
            df,
            how = "left",
            on = "donor_id",
            suffixes = [ "_x", "_y" ],
        )
    ))
    # This filter prevents us from double-counting candidate-combinations
    .loc[lambda df: df["candidate_x"] < df["candidate_y"]]
    .sort_values([
        "candidate_x",
        "candidate_y",
        "donor_id"
    ])
)

candidate_pairs.head(10)
```

## Main tables and auxilary tables